In [69]:
import plotly.plotly as py
#import plotly.offline as py
from plotly.graph_objs import *
import plotly
import networkx as nx
import numpy as np
import random as rnd
plotly.tools.set_credentials_file(username='your_username', api_key='your_api_key')
#create your credentials into https://plot.ly/#/

# Set graph and your properties

In [70]:
G=nx.read_graphml('graphG.graphml')

In [71]:
nodes = G.nodes(data=True)
x = nx.get_node_attributes(G,"longitude")
y = nx.get_node_attributes(G,"latitude")
z = nx.get_node_attributes(G,"zaxis")
pos = x
for i in range(len(x)):
    j = nodes[i][0]
    pos[j]=(x[j],y[j],z[j])
nx.set_node_attributes(G,'pos',pos)

In [72]:
#set center lattice cubic
dmin=1
ncenter=0
for n in pos:
    x,y,z=pos[n]
    d=(x-0.5)**2+(y-0.5)**2+(z-0.5)**2
    if d<dmin:
        ncenter=n
        dmin=d

In [62]:
p=nx.single_source_shortest_path_length(G,ncenter)
edge_trace = Scatter3d(
    x=[],
    y=[],
    z=[],
    line=Line(width=2.0,color='red'),
    #line=Line(width=1.0,color='#888'),
    hoverinfo='none',
    mode='lines')

# Determine SCC clusters

In [73]:
scc = sorted(nx.strongly_connected_components(G),key=len,reverse=True)
nodes = G.nodes()
component = nx.get_node_attributes(G,'x')
colors = []
opacities = []
order=0
for i in scc:
    for j in list(i):
        idn = j
        component[idn]=order
        if(order>0):
            colors.append("gray")
            opacities.append(1.0)
        else:
            colors.append("blue")
            opacities.append(0.5)
    order+=1
nx.set_node_attributes(G,'component',component)

In [74]:
for edge in G.edges():
    x0, y0, z0 = G.node[edge[0]]['pos']
    x1, y1, z1 = G.node[edge[1]]['pos']
    edge_trace['x'] += [x0, x1, None]
    edge_trace['y'] += [y0, y1, None]
    edge_trace['z'] += [z0, z1, None]

In [75]:
node_trace = Scatter3d(
    x=[],
    y=[],
    z=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=Marker(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='Blackbody',
        reversescale=True,
        color=colors,
        opacity=0.5,
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=0)))

In [66]:
for node in G.nodes():
    x, y, z = G.node[node]['pos']
    node_trace['x'].append(x)
    node_trace['y'].append(y)
    node_trace['z'].append(z)

In [67]:
for n in G.nodes():
    #node_trace['marker']['color'].append(len(adjacencies))
    #node_info = '# of connections: '+str(len(adjacencies))
    #node_trace['text'].append(node_info)
    node_trace['marker']['color'].append(component[n])
    node_info = 'id component: '+str(component[n])
    node_trace['text'].append(node_info)


In [68]:
fig = Figure(data=Data([edge_trace, node_trace]),
             layout=Layout(
                title='<br>Network graph made with Python',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))

py.iplot(fig, filename='networkx')